In [1]:
import cv2
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from covidxpert import load_image, perspective_correction
from covidxpert.perspective_correction.add_padding import add_padding
from scipy.signal import find_peaks

ModuleNotFoundError: No module named 'covidxpert.perspective_correction.add_padding'

In [ ]:
def get_kernel_size(image:np.ndarray)->int:
    kernel_size = int(min(image.shape) / 5)
    
    if kernel_size % 2 == 0:
        kernel_size += 1
        
    return kernel_size

In [ ]:
def apply_mean_blur(image:np.ndarray)->np.ndarray:
    return cv2.medianBlur(image, get_kernel_size(image))

In [ ]:
def apply_median_threshold(image:np.ndarray)->np.ndarray:
    return cv2.threshold(image, np.median(image)/2, 255, cv2.THRESH_BINARY)[1]

In [ ]:
def trim_padding(image:np.ndarray, padding:int)->np.ndarray:
    return image[padding:-padding, padding:-padding]

In [ ]:
def count_from_left_side(mask:np.ndarray):
    counter = 0
    for boolean in mask:
        if boolean:
            counter += 1
        else:
            break
    return counter

In [ ]:
def count_from_right_side(mask:np.ndarray):
    return count_from_left_side(np.flip(mask, axis=0))

In [ ]:
def build_slice(left:int, right:int, maximum:int):
    return slice(left, maximum if right==0 else right)

In [ ]:
def strip_black(image:np.ndarray, mask:np.ndarray, v_threshold:float, h_threshold:float)->np.ndarray:
    vertical_mask = mask.mean(axis=1) < v_threshold
    horizzontal_mask = mask.mean(axis=0) < h_threshold
    
    h_slice = build_slice(
        count_from_left_side(horizzontal_mask),
        -count_from_right_side(horizzontal_mask),
        image.shape[1]
    )
    v_slice = build_slice(
        count_from_left_side(vertical_mask),
        -count_from_right_side(vertical_mask),
        image.shape[0]
    )
    return image[v_slice, h_slice]

In [ ]:
def compute_median_threshold(mask:np.ndarray)->float:
    masked_mask = strip_black(mask, mask, 0, 0)
    v_white_median = np.median(masked_mask.mean(axis=0))
    h_white_median = np.median(masked_mask.mean(axis=1))
    return v_white_median/2, h_white_median/2

In [ ]:
def normalize_image(image:np.ndarray)->np.ndarray:
    return np.uint8(((image - image.min()) / (image.max() - image.min()))*255)

In [ ]:
def remove_artefacts(image:np.ndarray, sigma_1:float=0.3, sigma_2:float=15)->np.ndarray:
    blur = cv2.GaussianBlur(image, (5,5), sigma_2, sigma_2).astype(float)
    result = image - blur
    result = normalize_image(image)
    result = cv2.threshold(result, np.median(result), 255, cv2.THRESH_BINARY)[1]
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(result, connectivity=8)
    sizes = stats[1:, -1]
    area = result.shape[0]*result.shape[1]
    artefacts = np.zeros(result.shape, dtype=bool)
    
    for i, size in enumerate(sizes):
        if size < area/10:
            artefacts |= output==i+1
    
    kernel = np.ones((3,3),np.uint8)

    artefacts = cv2.dilate(artefacts.astype(np.uint8), kernel).astype(bool)

    cleared_image = image.copy()
    cleared_image[artefacts]=0

    return cleared_image, artefacts

In [ ]:
def get_blur_mask(image:np.ndarray, padding:int):
    blurred = add_padding(image, padding)
    blurred, _ = remove_artefacts(blurred)
    blurred = apply_mean_blur(blurred)
    blurred = apply_median_threshold(blurred)
    return trim_padding(blurred, padding)

In [ ]:
def blur_bbox(image:np.ndarray, padding:int=50)->np.ndarray:
    mask = get_blur_mask(image, padding)
    return strip_black(image, mask, *compute_median_threshold(mask))

In [ ]:
paths = glob("tests/test_images/*.jpg")

In [ ]:
from skimage.morphology import disk
from skimage.filters import rank

In [ ]:
for path in paths:
    original = load_image(path)
    image = perspective_correction(original)
    cleared_image, artefacts = remove_artefacts(image)
    bboxed = blur_bbox(image)
    
    fig, axes = plt.subplots(ncols=5, figsize=(25, 5))
    axes[0].imshow(original, cmap="gray")
    axes[1].imshow(artefacts, cmap="gray")
    axes[2].imshow(cleared_image, cmap="gray")
    axes[3].imshow(image, cmap="gray")
    axes[4].imshow(bboxed, cmap="gray")
    plt.show()